In [12]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table
from dash.dependencies import Input, Output
from bson.json_util import dumps,loads
import base64
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pymongo import MongoClient

#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from AnimalShelter import AnimalShelter





###########################
# Data Manipulation / Model
###########################
# FIX ME change for your username and password and CRUD Python module name
username = "aacuserr"
password = "mason"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object 
df =pd.DataFrame.from_records(list(shelter.read({})))



#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'coco.jpg' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard'))),
    html.Div(className="header",
             children=[html.Center(html.Img(src="https://i.stack.imgur.com/qhYox.png",width="500", height="600")), #image 
                      html.Center(html.B(html.H6('Mason Voorhees')))]), #Unique Handle
    html.Br(),
    html.Div(className="row",
            children=[
                html.Button(id='submit-button-one',n_clicks=0, children='Cats'),
                html.Button(id='submit-button-two',n_clicks=0,children='Dogs') ]),
    
    html.Div(dash_table.DataTable(
            id='datatable-interactivity',
                columns=[
                    {"name": i, "id": i,"age_upon_outcome":i, 'animal_id':i,'animal_type':i,'breed':i,"color":i,
                     'date_of_birth':i,'datetime':i,'monthyear':i,'outcome_subtype':i,'outcome_type':i,'sex_upon_outcome':i,
                     'location_lat':i,'location_long':i,'age_upon_outcome_in_weeks':i,
                    "deletable": False, "selectable": True} for i in df.columns]
                    ,data=df.to_dict('records'),editable = False,filter_action='native')),
     html.Br(),  
     html.Div(
            dcc.Graph(            #chart
                id ='graph_id',
                figure=list(shelter.read({}))
                           ),
            title="Outcome_Type"),
       
      html.Br(),
      html.Div(   #location map
            id='map-id',
            className="col s12 m6",
            title="Location"
            )]) 
            
        
       

 ##Interaction Between Components / Controller

#This callback will highlight a row on the data table when the user selects it
@app.callback(
    Output('datatable-interactivity',"data"),
    [Input('submit-button-one','n_clicks'),
    Input('submit-button-two','n_clicks')
    ]
)
def on_click(bt1, bt2):
    if(int(bt1) ==0 and int(bt2) ==0):
        df =pd.DataFrame.from_records(shelter.read({}))
    elif (int(bt1) > int(bt2)):
            df = pd.DataFrame(dumps(shelter.read({"animal__type":"Cat"})))
            df =pd.DataFrame(dumps(shelter.read({"animal_type":"Dog"})))
    return df.to_dict('records')
@app.callback(
    Output('datatable-interactivity', 'style_data_conditional'),
    [Input('datatable-interactivity', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback(
    Output('graph_id','figure'),
    [Input('datatable-interactivity',"derived_virtual_data")])
def update_Graph(viewData):
    dff = pd.DataFrame.from_dict(viewData)
    pieChart = px.pie(
        data_frame=dff,
        names=dff['outcome_type'],
        hole = 3,
    )
    return pieChart
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-interactivity', "derived_viewport_data")])
def update_map(viewData):
    dff = pd.DataFrame.from_dict(viewData)
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            dl.Marker(position=[30.75,-97.48], children=[
                dl.Tooltip(dff.iloc[0,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[1,9])
                ])
            ])
        ])
    ]    

app

hello
